# External Lab 

Here each question is of 1 mark.

# Multiple Linear Regression

## Problem Statement

Use Multiple Linear Regression to **predict the consumption of petrol** given relevant variables are the petrol tax, the per capita, income, the number of miles of paved highway, and the proportion of the population with driver's licenses.

## Dataset

There are 48 rows of data.  The data include:

      I,  the index;
      A1, the petrol tax;
      A2, the per capita income;
      A3, the number of miles of paved highway;
      A4, the proportion of drivers;
      B,  the consumption of petrol.

### Reference 

    Helmut Spaeth,
    Mathematical Algorithms for Linear Regression,
    Academic Press, 1991,
    ISBN 0-12-656460-4.

    S Weisberg,
    Applied Linear Regression,
    New York, 1980, pages 32-33.

## Question 1 - Exploratory Data Analysis

*Read the dataset given in file named **'petrol.csv'**. Check the statistical details of the dataset.*

**Hint:** You can use **df.describe()**

In [213]:
import numpy as np
import pandas as pd

df = pd.read_csv("petrol-1.csv");

In [214]:
df.describe()

,tax,income,highway,dl,consumption
count,48.000000,48.000000,48.000000,48.000000,48.000000
mean,7.668333,4241.833333,5565.416667,0.570333,576.770833
std,0.950770,573.623768,3491.507166,0.055470,111.885816
min,5.000000,3063.000000,431.000000,0.451000,344.000000
25%,7.000000,3739.000000,3110.250000,0.529750,509.500000
50%,7.500000,4298.000000,4735.500000,0.564500,568.500000
75%,8.125000,4578.750000,7156.000000,0.595250,632.750000
max,10.000000,5342.000000,17782.000000,0.724000,968.000000


# Question 2 - Cap outliers 

Find the outliers and cap them. (Use (Q1 - 1.5 * IQR) as the minimum cap and (Q3 + 1.5 * IQR) as the max cap. The decision criteria is you should consider the datapoints which only falls within this range. The data points which fall outside this range are outliers and the entire row needs to be removed

In [215]:
capDf = pd.DataFrame()
for col in df.columns:
    percentiles = df[col].quantile([0.25,0.75]).values
    percentiles[0] = percentiles[0] - (percentiles[1] - percentiles[0]) * 1.5
    percentiles[1] = percentiles[1] + (percentiles[1] - percentiles[0]) * 1.5
    print(percentiles[0], percentiles[1])
    capDf[col] = df[col].loc[(df[col] >= percentiles[0]) & (df[col] <= percentiles[1])]

capDf

5.3125 12.34375
2479.375 7727.8125
-2958.375 22327.5625
0.4315000000000002 0.8408749999999996
324.625 1094.9375


,tax,income,highway,dl,consumption
0,9.00,3571,1976,0.525,541
1,9.00,4092,1250,0.572,524
2,9.00,3865,1586,0.580,561
3,7.50,4870,2351,0.529,414
4,8.00,4399,431,0.544,410
5,10.00,5342,1333,0.571,457
6,8.00,5319,11868,0.451,344
7,8.00,5126,2138,0.553,467
8,8.00,4447,8577,0.529,464
9,7.00,4512,8507,0.552,498


# Question 3 - Independent variables and collinearity 
Which attributes seems to have stronger association with the dependent variable consumption?

In [216]:
capDf.corr()

,tax,income,highway,dl,consumption
tax,1.000000,-0.009085,-0.395852,-0.321635,-0.459430
income,-0.009085,1.000000,0.089118,0.156699,-0.241797
highway,-0.395852,0.089118,1.000000,-0.067926,-0.028022
dl,-0.321635,0.156699,-0.067926,1.000000,0.702413
consumption,-0.459430,-0.241797,-0.028022,0.702413,1.000000


In [217]:
#There is a stronger association between drivers and consumption as the correlation coefficient is higher

### Observing the above correlation values between all the variables, we can see that there is stronger association between the number of drivers and consumption. And comparatively tax has an association in a negative way. 
Insights :
As tax increases the consumption decreases.
As number of drivers is more consumption is more

# Question 4 - Transform the dataset 
Divide the data into feature(X) and target(Y) sets.

In [218]:
input_vars_X = pd.DataFrame()
target_vars_Y = pd.DataFrame()
input_vars_X = capDf.iloc[ : , [0,3]]
target_vars_Y = capDf.iloc[:, 4:]

# Question 5 - Split data into train, test sets 
Divide the data into training and test sets with 80-20 split using scikit-learn. Print the shapes of training and test feature sets.

In [219]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input_vars_X, target_vars_Y, test_size=0.20, random_state=1)
print("Shape X-Train:", x_train.shape)
print("Shape X-Test:", x_test.shape)
print("Shape Y-Train:", y_train.shape)
print("Shape Y-Test:", y_test.shape)

Shape X-Train: (37, 2)
Shape X-Test: (10, 2)
Shape Y-Train: (37, 1)
Shape Y-Test: (10, 1)


# Question 6 - Build Model 
Estimate the coefficients for each input feature. Construct and display a dataframe with coefficients and X.columns as columns

In [220]:
model = LinearRegression()
model.fit(x_train, y_train)
beta1 = model.coef_
print("Coef (b1): ", beta1)
coeffDf = pd.DataFrame(np.array(beta1).tolist(), columns = input_vars_X.columns)
coeffDf


Coef (b1):  [[ -42.66667573 1312.54863797]]


,tax,dl
0,-42.666676,1312.548638


# R-Square 

# Question 7 - Evaluate the model 
Calculate the accuracy score for the above model.

In [221]:
model.score(input_vars_X, target_vars_Y)

0.544562593274917

# Question 8: Repeat the same Multi linear regression modelling by adding both Income and Highway features
Find R2 


In [222]:
input_vars_X_2 = pd.DataFrame()
target_vars_Y_2 = pd.DataFrame()
input_vars_X_2 = capDf.iloc[ : , 0:4]
target_vars_Y_2 = capDf.iloc[:, 4:]

In [223]:
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(input_vars_X_2, target_vars_Y_2, test_size=0.20, random_state=1)
print("Shape X-Train:", x_train_2.shape)
print("Shape X-Test:", x_test_2.shape)
print("Shape Y-Train:", y_train_2.shape)
print("Shape Y-Test:", y_test_2.shape)
model2 = LinearRegression()
model2.fit(x_train_2, y_train_2)

Shape X-Train: (37, 4)
Shape X-Test: (10, 4)
Shape Y-Train: (37, 1)
Shape Y-Test: (10, 1)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [224]:
model2.score(input_vars_X_2, target_vars_Y_2)

0.6611572397634741

# Question 9: Print the coefficients of the multilinear regression model

In [225]:
beta1_2 = model2.coef_
print("Coef (b1): ", beta1_2)

Coef (b1):  [[-4.34829669e+01 -7.31687150e-02 -3.04888622e-03  1.44101372e+03]]


In [226]:
coeffDf_2 = pd.DataFrame(np.array(beta1_2).tolist(), columns = input_vars_X_2.columns)
coeffDf_2

,tax,income,highway,dl
0,-43.482967,-0.073169,-0.003049,1441.013715


# Question 10 
In one or two sentences give reasoning on R-Square on the basis of above findings
Answer

### *R squared value increase if we increase the number of independent variables to our analysis

In [227]:
#Based on the above results, we observe that model accuracy i.e. R^2 increases as we supplement our model with
#additional independent variables. This may be due to availability of more data points to predict the 
#consumption of petrol